In [ ]:
from models.actor_critic import PPOActorCritic
import gymnasium as gym
import numpy as np
import torch
from itertools import count

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env = gym.make('Pendulum-v1', render_mode='human')
ac = PPOActorCritic(in_features=4).to(device)
ac.load_state_dict(torch.load("saved_models/ac_model.pth"))
ac.eval()

num_episodes = 30

for episode in range(num_episodes):
    states, info = env.reset()
    for t in count():
        states = torch.from_numpy(states).float().to(device)
        t_feature = torch.tensor([t], device=device)
        states = torch.cat((states, t_feature), dim=-1)

        with torch.no_grad():
            action, log_prob, pre_squashed_action, state_values = ac.get_action_and_value(states, deterministic=True)

        new_states, rew, terminated, truncated, info = env.step(action.to('cpu').numpy())
        states = new_states

        if truncated or terminated:
            break

/tmp/ipykernel_4484/3183483101.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ac.load_state_dict(torch.load("saved_models/ac_model.pth"))


KeyboardInterrupt: 

In [8]:
env.close()